# ORM Has Many Reading

### Introduction

In this lesson, we'll fill in our other relationship methods.  We'll see how we can query our database fill in our `hasmany` functions.  For example, we'll stick with the moe's bar domain, but this time fill in a query that returns the many orders that each customer may have.

### Loading up our database

Let's again work with Moe's bar.  We can begin by loading the data from our database.  And let's take a look at the tables.  

> This lesson will work if you download it from github.

In [1]:
import sqlite3
conn = sqlite3.connect('./moes_bar.db')
cursor = conn.cursor()

import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-through-bar/data/"
names = ['bartenders', 'customers', 'drinks', 'orders', 'ingredients', 'ingredients_drinks']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False, if_exists = 'replace')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('bartenders',),
 ('customers',),
 ('drinks',),
 ('orders',),
 ('ingredients',),
 ('ingredients_drinks',)]

### Has many queries

Now let's get moving with our has many relationship.  We already saw how to add an method to our order class to return the associated customer.

In [2]:
class Order:
    __table__ = 'orders'
    columns = ['id', 'customer_id', 'drink_id', 'bartender_id']

    def customer(self):
        cursor.execute('SELECT * FROM customers WHERE id = ?', (self.customer_id,))
        customer_record = cursor.fetchone()
        return build_from_record(Customer, customer_record)

class Customer:
    __table__ = 'customers'
    columns = ['id', 'name', 'hometown']


def build_from_record(Class, record):
    if not record: return None
    attr = dict(zip(Class.columns, record))
    obj = Class()
    obj.__dict__ = attr
    return obj

def build_from_records(Class, records):
    return [build_from_record(Class, record) for record in records]

Now let's add a method to our customer class that returns the all of the related orders.  Once again, it's easier if we start with a record from the database.

In [3]:
cursor.execute('SELECT * FROM customers ORDER BY id DESC LIMIT 1;')
last_customer_record = cursor.fetchone()

last_customer = build_from_record(Customer, last_customer_record)

last_customer.__dict__

{'id': 3, 'name': 'lisa simpson', 'hometown': 'philly'}

So now, starting with this customer let's think about how we can get all of the associated orders.  Well the only id we have is the customer id.  And we can use that if we remember that there is a `customer_id` column on the orders table.  Our method looks like the following:

In [4]:
class Customer:
    __table__ = 'customers'
    columns = ['id', 'name', 'hometown']

    def orders(self):
        cursor.execute('SELECT * FROM orders WHERE orders.customer_id = ?;', (self.id,))
        order_records = cursor.fetchall()
        return build_from_records(Order, order_records)

In [5]:
last_customer = build_from_record(Customer, last_customer_record)

In [6]:
orders = last_customer.orders()
orders

[<__main__.Order at 0x7a96fd8a6ec0>, <__main__.Order at 0x7a96d2416b60>]

> Take a moment to read back through the query and make sure that you understand how it works.

### Give it a shot

In fact if we give you the Bartender class, add a function that returns all of the bartender's orders.

In [7]:
class Bartender:
    __table__ = 'bartenders'
    columns = ['id', 'name', 'hometown', 'birthyear']

    def orders(self):
        query = 'select * from orders where orders.bartender_id = ?'
        cursor.execute(query, self.id)
        records = cursor.fetchall()
        return build_from_records(Order, records)

### Many to Many

Finally, the last relationship to tackle is the many to many relatioship.  For example, a customer has many bartenders and a bartender has many customers.  These two entities are linked through our join table, here `orders`.  Let's think about what it would take for us to add an `bartenders` function to our customers table.

In [ ]:
last_customer.__dict__

{'id': 3, 'name': 'lisa simpson', 'hometown': 'philly'}

Well, the only thing useful on our customer is the customer id.  We can use this by finding all of the orders whose customer_id matches the id on customer, and then getting the associated bartenders of those orders.  Ok, let's give it a shot.

In [ ]:
class Customer:
    __table__ = 'customers'
    columns = ['id', 'name', 'hometown']

    def orders(self):
        cursor.execute('SELECT * FROM orders WHERE orders.customer_id = ?;', (self.id,))
        order_records = cursor.fetchall()
        return build_from_records(Order, order_records)

    def bartenders(self):
        cursor.execute("""SELECT DISTINCT bartenders.*
        FROM bartenders JOIN orders ON
        orders.bartender_id = bartenders.id
        WHERE orders.customer_id =  ?;""", (self.id,))
        bartender_records = cursor.fetchall()
        return build_from_records(Bartender, bartender_records)

In [ ]:
last_customer = build_from_record(Customer, last_customer_record)

In [ ]:
bartenders = last_customer.bartenders()

[bartender.__dict__ for bartender in bartenders]

[{'id': 3, 'name': 'patty', 'hometown': 'philly', 'birthyear': 1970}]

In [10]:
class Bartender:
    __table__ = 'bartenders'
    columns = ['id', 'name', 'hometown', 'birthyear']

    def orders(self):
      query = 'select * from orders where orders.bartender_id = ?'
      cursor.execute(query, self.id)
      records = cursor.fetchall()
      return build_from_records(Order, records)

    def customers(self):
      query = """select distinct c.* from orders o
      JOIN customers c ON o.customer_id = c.id
      WHERE o.bartender_id = ?""";
      cursor.execute(query, self.id)
      records = cursor.fetchall()
      return build_from_records(Customer, records)


### Summary

In this lesson, we'll saw how to write a has many query and a many to many query.  With both queries, we start with the primary key, like `customers.id`, and find the relevant records on a separate table.  For example, to find the customers orders we wrote:

In [ ]:
class Customer:
    __table__ = 'customers'
    columns = ['id', 'name', 'hometown']

    def orders(self):
        cursor.execute('SELECT * FROM orders WHERE orders.customer_id = ?;', (self.id,))
        order_records = cursor.fetchall()
        return build_from_records(Order, order_records)

So when we call `customer.orders()`, we find the orders whose `customer_id` matches the id of the current `customer`.

With our many to many relationship, we this time joined two different tables together.  We saw this with our `customer.bartenders()` function.

In [ ]:
def bartenders(self):
        cursor.execute("""SELECT DISTINCT bartenders.*
        FROM bartenders JOIN orders ON
        orders.bartender_id = bartenders.id
        WHERE orders.customer_id =  ?;""", (self.id,))
        bartender_records = cursor.fetchall()
        return build_from_records(Bartender, bartender_records)

We needed to return the relevant bartender records from our bartenders table, but found these records by finding the orders whose `customer_id` equaled our current customer's id.